In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"

/Users/csuarezgurruchaga/Desktop/Digital-House/clase_12/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Introducción

Data wrangling es el proceso de limpieza y unificación de conjuntos de datos desordenados y complejos para facilitar su acceso, exploración, análisis o modelización posterior.

Las tareas que involucra son
* Limpieza de datos
* Eliminación de registros duplicados
* Transformación de datos
* Discretización de variables
* Detección y filtro de outliers
* Construcción de variables dummies

Pandas provee métodos para llevar a cabo estas tareas, y en esta práctica repasaremos algunos de estos métodos.

## Dataset

En esta clase usaremos un dataset con info de películas que disponibiliza datos de movielens (https://movielens.org/).

https://grouplens.org/datasets/movielens/

http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

Este conjunto de datos está conformado por varios archivos:
* **movies**: idPelicula, título y género; 

donde cada registro tiene los datos de una película

* **ratings**: idUsuario, idPelicula, rating, fecha; 

donde cada registro tienen la calificación otorgada por un usuario a una película

* **tags**: idUsuario, idPelicula, tag, fecha; 

donde cada registro tienen el tag que asignó un usuario a una película

## Imports

In [2]:
import pandas as pd
import numpy as np
import re

## Ejercicio 1 - Importar datos

Leamos los datos de movies, ratings y tags desde los archivos
* ../Data/movies.csv
* ../Data/ratings.csv
* ../Data/tags.csv

en las variables 
* data_movies
* data_ratings
* data_tags

Veamos cuántos registros hay en cada DataFrame y de qué tipos son los datos de cada columna. 

Veamos los primeros registros de cada DataFrame para verificar que los datos fueron importados correctamente.

In [3]:
path1='../Data/movies.csv'
path2='../Data/ratings.csv'
path3='../Data/tags.csv'

In [4]:
df1=pd.read_csv(path1, sep=',')
df2=pd.read_csv(path2, sep=',')
df3=pd.read_csv(path3, sep=',')

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [7]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


## Ejercicio 2 - duplicated, drop_duplicates

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html

**2.a** Analicemos los ratings asignados por los usuarios a las películas, y detectemos aquellos usuarios que calificaron más de una película.

**2.b** Veamos si hay usuarios que calificaron más de una vez a la misma película

**2.c** Creemos un dataframe donde cada usuario haya calificado exactamente una película

In [8]:
df2[df2.duplicated('userId')]['userId']

1           1
2           1
3           1
4           1
5           1
         ... 
100831    610
100832    610
100833    610
100834    610
100835    610
Name: userId, Length: 100226, dtype: int64

In [9]:
df2[df2.duplicated(['userId','movieId'])].any()

userId       False
movieId      False
rating       False
timestamp    False
dtype: bool

In [10]:
df2.drop_duplicates('userId')

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
232,2,318,3.0,1445714835
261,3,31,0.5,1306463578
300,4,21,3.0,986935199
516,5,1,4.0,847434962
...,...,...,...,...
97364,606,1,2.5,1349082950
98479,607,1,4.0,964744033
98666,608,1,2.5,1117408267
99497,609,1,3.0,847221025


## Ejercicio 3 - replace

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html

El campo `genre` del DataFrame data_movies tiene una lista de géneros separadas por el caracter "|"

**3.a** Una lista de valores viejos por un valor nuevo

Vamos a buscar los valores de los registros que tengan "Fantasy" como alguno sus géneros y reemplazarlos por "Fantasy"

**3.b** Una lista de valores viejos por una lista de valores nuevos

Tomamos 10 valores del campo género al azar y construimos una lista con ellos, y otra con el último género que figura en cada elemento de la primera lista. Usamos esas dos listas para reemplazar unos valores por otros.

**3.c** Un diccionario que defina los mapeos

Con las dos listas del paso anterior, construimos un diccionario que pasamos como argumento al método replace para definir algunos reemplazos.


In [11]:
patron = re.compile(r'Fantasy')
fantasy= df1.genres.apply(lambda x: re.search(patron,str(x)))
fantasy_mask = fantasy.notnull()

valores_unicos_a_reemplazar=list(df1[fantasy_mask]['genres'].unique())

data_genres_3a = df1.replace(valores_unicos_a_reemplazar,'Fantasy')
data_genres_3a

,movieId,title,genres
0,1,Toy Story (1995),Fantasy
1,2,Jumanji (1995),Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Fantasy
9738,193583,No Game No Life: Zero (2017),Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [12]:
rng = np.random.default_rng()
row = rng.integers(low=0, high=df1.shape[0], size=10)

valores=list(df1.loc[row,'genres'])

In [13]:
generos_a_reemplazar = []
for valor in valores :
    asd = (valor).split('|')
    posicion_final = len(asd)-1
    generos_a_reemplazar.append(asd[posicion_final])
generos_a_reemplazar

['Horror',
 'Horror',
 'Horror',
 'Romance',
 'Horror',
 'Thriller',
 'Drama',
 'War',
 'Western',
 'Mystery']

In [14]:
data_genres_3b = df1.replace(valores,generos_a_reemplazar)
data_genres_3b

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [15]:
generos_a_reemplazar

['Horror',
 'Horror',
 'Horror',
 'Romance',
 'Horror',
 'Thriller',
 'Drama',
 'War',
 'Western',
 'Mystery']

In [16]:
valores

['Horror',
 'Horror',
 'Horror',
 'Comedy|Romance',
 'Horror',
 'Action|Crime|Thriller',
 'Drama',
 'Drama|Thriller|War',
 'Action|Thriller|Western',
 'Comedy|Drama|Mystery']

In [17]:
dictionary = dict(zip(valores,generos_a_reemplazar))


data_genres_3c = df1.replace(dictionary)
data_genres_3c

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


## Ejercicio 4 - cut

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html

El método `cut` nos permite asignar categorías a intervalos numéricos.

Creemos la columna de categorías (rating_label_cut) asociadas al campo rating de data_ratings de este modo:

* mala, para puntajes menores a 3;

* regular, para mayor igual a 3 y  menor que 4;

* buena para puntaje mayor o igual que 4


In [18]:

rating_label_cut=pd.cut(df2.rating, bins=[0,3,4,5.1],labels = ["mala","regular","buena"], right=False)
df2['rating_label_cut'] = rating_label_cut
df2.sample(10)

,userId,movieId,rating,timestamp,rating_label_cut
91284,591,3174,3.0,970525001,regular
10092,66,1302,4.5,1113188364,buena
19919,132,1,2.0,1157921785,mala
18636,119,60072,3.0,1436007571,regular
9764,64,4016,4.5,1161564478,buena
43909,292,103772,3.5,1421974631,regular
38559,265,1270,5.0,965314811,buena
41081,279,608,2.0,1506394168,mala
50264,325,1207,4.0,1039397109,buena
92662,599,76,2.5,1498518457,mala


##  Ejercicio 5 - quantile

Calculemos los cuartilos del campo timestamp de la tabla data_ratings y asociemos las etiquetas:
* muy antigua, para el primer cuartilo
* antigua, para el segundo cuartilo
* pasada, para el tercer cuartilo
* reciente, para el cuarto cuartilo

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.quantile.html

In [19]:
q1 = df3.timestamp.quantile(0.25)
q2 = df3.timestamp.quantile(0.5)
q3 = df3.timestamp.quantile(0.75)
q4 = df3.timestamp.quantile(1)

timestamp_categories=pd.cut(df3.timestamp,bins = [0,q1,q2,q3,q4],labels=['muy antigua','antigua','pasada','reciente'], right=False)
df3['timestamp_categories']=timestamp_categories
df3.head(10)

,userId,movieId,tag,timestamp,timestamp_categories
0,2,60756,funny,1445714994,pasada
1,2,60756,Highly quotable,1445714996,pasada
2,2,60756,will ferrell,1445714992,pasada
3,2,89774,Boxing story,1445715207,pasada
4,2,89774,MMA,1445715200,pasada
5,2,89774,Tom Hardy,1445715205,pasada
6,2,106782,drugs,1445715054,pasada
7,2,106782,Leonardo DiCaprio,1445715051,pasada
8,2,106782,Martin Scorsese,1445715056,pasada
9,7,48516,way too long,1169687325,antigua


##  Ejercicio 6 - get_dummies

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

Vamos a crear ahora variables dummy para representar el campo timestamp_category del DataFrame data_ratings, y vamos a reemplazar ese campo por las variables dummies.


In [21]:
dummies_timestamp_category = pd.get_dummies(df3.timestamp_categories, prefix='timestamp_categories', drop_first=True)
dummies_timestamp_category

joined=df3.join(dummies_timestamp_category)

joined.drop('timestamp_categories', axis=1, inplace=True)
joined

,userId,movieId,tag,timestamp,timestamp_categories_antigua,timestamp_categories_pasada,timestamp_categories_reciente
0,2,60756,funny,1445714994,0,1,0
1,2,60756,Highly quotable,1445714996,0,1,0
2,2,60756,will ferrell,1445714992,0,1,0
3,2,89774,Boxing story,1445715207,0,1,0
4,2,89774,MMA,1445715200,0,1,0
...,...,...,...,...,...,...,...
3678,606,7382,for katie,1171234019,1,0,0
3679,606,7936,austere,1173392334,1,0,0
3680,610,3265,gun fu,1493843984,0,1,0
3681,610,3265,heroic bloodshed,1493843978,0,1,0
